In [2]:
import re

def parse_percentage(my_list):
    return [int(int(element.strip('%'))*255/100) if element != 'FL' else 255 for element in my_list]

    # my_list = [element if element != 'FL' else "0" for element in my_list]
    # return [255 if int(element.strip('%')) * 255 / 100 > 120 else 0 for element in my_list]

# Replace 'your_file.txt' with the path to your text file
file_path = 'capture.txt'


# Regular expressions to extract frame number, universe, and DMX channels
frame_number_pattern = r"Frame (\d+):"
universe_pattern = r"Universe: (\d+)"
dmx_channels_pattern = r"DMX Channels\n((?:.+\n)+)"
# Open the file

silver_data = []
with open(file_path, 'r') as file:
    # Read and print each line
    file_content = file.read()
    splitted_file = file_content.split("No.     Time           Text       ID         Frame length on the wire Extended Flag Time since reference or first frame Frame-Length Frame length stored into the capture file Time delta from previous captured frame Epoch Time Padding")
    print(len(splitted_file))
    for text in splitted_file[1:] :
      # print(text)
      # packet = packet_noise.split("No.")[0]
      # print(packet)

      time = float(text[9:].split(" ")[0])
      # Extract frame number
      frame_number_match = re.search(frame_number_pattern, text)
      frame_number = int(frame_number_match.group(1)) if frame_number_match else None

      # Extract universe
      universe_match = re.search(universe_pattern, text)
      universe = int(universe_match.group(1)) if universe_match else None

      # Extract DMX channels
      dmx_channels_match = re.search(dmx_channels_pattern, text)
      dmx_channels = dmx_channels_match.group(1) if dmx_channels_match else None

      # Store the extracted information in a dictionary
      data = {
          "time" : time,
          "frame_number": frame_number,
          "universe": universe,
          "dmx_channels": dmx_channels.strip() if dmx_channels else None
      }
      # print(data)
      # print(len(data['dmx_channels'].split("\n")))

      # # Splitting DMX channels by '\n'
      # dmx_channels_list = dmx_channels.split('\n')

      # # Extracting the first 10 channels for RED, the next 10 for GREEN, and the following 10 for BLUE
      # red_channels = dmx_channels_list[:10]
      # green_channels = dmx_channels_list[10:20]
      # blue_channels = dmx_channels_list[20:30]


      # data['RED'] = [parse_percentage(element.split(":")[1].strip().split()) for element in red_channels]
      # data['GREEN'] = [parse_percentage(element.split(":")[1].strip().split()) for element in green_channels]
      # data['BLUE'] = [parse_percentage(element.split(":")[1].strip().split()) for element in blue_channels]
      # # print(data['RED'])
      # # print(data['GREEN'])
      # # print(data['BLUE'])
      # print(data)
      # break
      silver_data.append(data)
print(len(silver_data))
# silver_data = sorted(silver_data, key=lambda x: x['frame_number'])
silver_data = sorted(silver_data, key=lambda x: x['time'])



1415
1414


In [3]:
universe_0_list = []
universe_1_list = []

for item in silver_data:
    if item['universe'] == 0:
        universe_0_list.append(item)
    elif item['universe'] == 1:
        universe_1_list.append(item)
# universe_0_list = sorted(universe_0_list, key=lambda x: x['frame_number'])
# universe_1_list = sorted(universe_1_list, key=lambda x: x['frame_number'])


In [4]:
gold_data = []
for i in range(0, len(universe_0_list)-1):
  data_uni_0 = universe_0_list[i]
  data_uni_1 = universe_1_list[i+1]


  dmx_channels_list = data_uni_0['dmx_channels'].split('\n')

  # Extracting the first 10 channels for RED, the next 10 for GREEN, and the following 10 for BLUE
  # red_channels = dmx_channels_list[:10]
  # green_channels = dmx_channels_list[10:20]
  # blue_channels = dmx_channels_list[20:30]

  red_channels = dmx_channels_list[0:30:3]
  green_channels = dmx_channels_list[1:30:3]
  blue_channels = dmx_channels_list[2:30:3]

  tmp_dict = {}
  tmp_dict['RED'] = [parse_percentage(element.split(":")[1].strip().split()) for element in red_channels]
  tmp_dict['GREEN'] = [parse_percentage(element.split(":")[1].strip().split()) for element in green_channels]
  tmp_dict['BLUE'] = [parse_percentage(element.split(":")[1].strip().split()) for element in blue_channels]





  dmx_channels_list = data_uni_1['dmx_channels'].split('\n')

  # Extracting the first 10 channels for RED, the next 10 for GREEN, and the following 10 for BLUE
  # red_channels = dmx_channels_list[:6]
  # green_channels = dmx_channels_list[6:12]
  # blue_channels = dmx_channels_list[12:18]

  red_channels = dmx_channels_list[0:18:3]
  green_channels = dmx_channels_list[1:18:3]
  blue_channels = dmx_channels_list[2:18:3]


  tmp_dict['RED'].extend([parse_percentage(element.split(":")[1].strip().split()) for element in red_channels])
  tmp_dict['GREEN'].extend([parse_percentage(element.split(":")[1].strip().split()) for element in green_channels])
  tmp_dict['BLUE'].extend([parse_percentage(element.split(":")[1].strip().split()) for element in blue_channels])


  gold_data.append({"RED" : tmp_dict['RED'], "GREEN" : tmp_dict['GREEN'], "BLUE" : tmp_dict['BLUE'], "FRAME": i})

In [6]:
len(gold_data)

706

In [8]:
import cv2
import numpy as np

# Define the dimensions of the frames
frame_width = 320
frame_height = 320

# Define the frame rate (FPS)
fps = 30

# Calculate the frame duration in milliseconds
frame_duration = int(1000 / fps)

# Create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output_video.avi', fourcc, fps, (frame_width, frame_height))

# Function to reverse every second row
def reverse_even_rows(matrix):
    for i in range(len(matrix)):
        if i % 2 == 1:
            matrix[i] = matrix[i][::-1]
    return matrix

# Iterate through each frame data
for frame_data in gold_data:
    # Create a blank frame
    frame = np.zeros((frame_height, frame_width, 3), dtype=np.uint8)

    # Upscale the color channels based on the frame data
    blue_channel = np.array(frame_data['BLUE'])
    green_channel = np.array(frame_data['GREEN'])
    red_channel = np.array(frame_data['RED'])

    # Reverse even rows
    green_channel = reverse_even_rows(green_channel)
    blue_channel = reverse_even_rows(blue_channel)
    red_channel = reverse_even_rows(red_channel)

    # Upscale to 320x320
    # frame[:, :, 0] = cv2.resize(blue_channel, (frame_width, frame_height), interpolation=cv2.INTER_NEAREST)
    frame[:, :, 1] = cv2.resize(green_channel, (frame_width, frame_height), interpolation=cv2.INTER_NEAREST)
    # frame[:, :, 2] = cv2.resize(red_channel, (frame_width, frame_height), interpolation=cv2.INTER_NEAREST)

    # Write the frame to the video
    out.write(frame)

# Release the VideoWriter object
out.release()


FCSC{L1ghtD3sign3rCr-gg!}